## Preparation

In [12]:
import torch, torchvision
from idlmav import MAV, plotly_renderer
model = torchvision.models.resnet18()
x = torch.randn(16,3,160,160)
mav = MAV(model, x, device='cpu')

## Portable figure
* Based on [plotly.graph_objects.Figure](https://plotly.com/python/creating-and-updating-figures/#figures-as-graph-objects)
* No dependency on `ipywidgets` or `plotly.graph_objects.FigureWidget` for portability reasons
* Displays correctly without the need of a running backend/kernel, e.g. in [nbviewer](https://nbviewer.org/)
* Interactions limited to plotly's built-in hover, pan and zoom mechanisms
* No synchronization between graph and table

In [13]:
with plotly_renderer('vscode'): mav.show_figure()

## Interactive widget
* Based on [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) and [plotly.graph_objects.FigureWidget](https://plotly.com/python/figurewidget/)
* Synchronizaton between slider, overview panel, main graph and table
  - Includes responsiveness of other components to plotly's built-in pan and zoom actions
* Clicking a node in the main graph highlights it in the table
* Limited portability expected to fluctuate over time on different environments

In [14]:
mav.show_widget(add_slider=True, add_overview=True)

## HTML export
* Most portable option
* Exports the same portable figure shown above to a standalone HTML file
* The `export_for_offline_use` parameter specifies how to include the plotly dependency in the exported HTML
  - `False` (default): The exported HTML is small, but requires a working internet connection to display correctly
  - `True`: The exported HTML is around 4MB in size and displays correctly without a working internet connection

In [15]:
mav.export_static_html('resnet18.html', export_for_offline_use=False)

## Specifying colors
* Palettes from plotly [discrete color sequences](https://plotly.com/python/discrete-color/#color-sequences-in-plotly-express) can be specified by name
* User-defined palettes can be specified as a list of `'#RRGGBB'` formatted strings
* The key to `fixed_color_map` may be a string in the **Operation** column or a category as listed [here](https://pytorch.org/docs/stable/nn.html)

In [16]:
with plotly_renderer('vscode'): mav.show_figure(
    palette='Vivid',
    avoid_palette_idxs=set([10]),
    fixed_color_map={'Convolution':7, 'add()':0, 'nn.MaxPool2d':5}
)

## Adding or removing panels
* This could help with portability or user experience on some environments, e.g.
  - On Colab the slider gets more in the way rather than adding value
  - The custom JS used for table synchronization may not be supported everywhere

In [17]:
mav.show_widget(add_overview=False, add_slider=False, add_table=False)

    'data': [{'hoverinfo': 'skip',
              'm…

## Modifying merging behaviour
* `merge_threshold<0` does not perform any merging
* `merge_threshold==0` only merges nodes that have zero parameters
* `merge_threshold` between 0 and 1 sorts nodes from the smallest to the largest by number of parameters and merges from the smallest node until just before the combined parameter count of merged nodes exceed the specified fraction of the total parameter count
* The following nodes are never merged:
  - Input and output nodes to the entire network
  - Nodes with multiple input connections
  - Nodes for which the input node has multiple output connections
* The default `merge_threshold` value normally results in nodes without parameters as well as normalization modules being merged

In [18]:
mav = MAV(model, x, device='cpu', merge_threshold=-1)
with plotly_renderer('vscode'): mav.show_figure(
    palette='Vivid',
    avoid_palette_idxs=set([10]),
    fixed_color_map={'Convolution':7, 'add()':0, 'nn.MaxPool2d':5}
)

## Calling internal components directly
* For users that wish to replace or augment one or more components
* A typical example would be replacing or subclassing the renderer to work on a specific environment

In [19]:
from idlmav import MavTracer, merge_graph_nodes, layout_graph_nodes, color_graph_nodes, WidgetRenderer
from IPython.display import display

tracer = MavTracer(model, x, device='cpu')
merge_graph_nodes(tracer.g)
layout_graph_nodes(tracer.g)
color_graph_nodes(tracer.g)
renderer = WidgetRenderer(tracer.g)
display(renderer.render())

    'data': [{'hoverinfo': 'skip',
              'm…

## Reducing notebook file size
* On some environments, plotly will include the entire plotly library (~ 4MB) in the notebook DOM for portable figures (`go.Figure`)
* This is not the case for interactive widgets (`go.FigureWidget`) where the plotly library is served from the backend
* Using a custom plotly renderer can also avoid this for `go.Figure`, importing plotly via a CDN instead
* Custom plotly renderers are made available in `idlmav` via a context manager:

In [20]:
from idlmav import plotly_renderer 
with plotly_renderer('notebook_connected'):
    mav.show_figure()

* Available custom plotly renderers may be listed as follows:

In [21]:
import plotly.io as pio
list(pio.renderers)

['plotly_mimetype',
 'jupyterlab',
 'nteract',
 'vscode',
 'notebook',
 'notebook_connected',
 'kaggle',
 'azure',
 'colab',
 'cocalc',
 'databricks',
 'json',
 'png',
 'jpeg',
 'jpg',
 'svg',
 'pdf',
 'browser',
 'firefox',
 'chrome',
 'chromium',
 'iframe',
 'iframe_connected',
 'sphinx_gallery',
 'sphinx_gallery_png']

* It is best to experiment with different renderers for your environment. From personal experience, the following may be good starting points:
  * `notebook_connected` or `vscode` with Plotly 5 on VSCode
  * `vscode` with Plotly 6 on VSCode
  * `iframe` with Plotly 5 on Kaggle